In [1]:
from sklearn import tree

# Fit decision tree
# need data_X_train, data_Y_train

clf = tree.DecisionTreeClassifier()
clf = clf.fit(data_X_train, data_Y_train)

# Evaluate on test dataset
# need data_X_test, data_Y_test

sum = 0
for i in range(np.arange(len(data_Y_test))):
  sum += (clf.predict(data_X_test[i]) == data_Y_test[i])

test_acc = sum / len(data_Y_test)
print(f"Testing accuracy for decision tree is {test_acc:.2f}")

# Plot decision tree

tree.plot_tree(clf)